# A360 Metadetection Shear

Contact author: Miranda Gorsuch

Create a shear profile for A360 using cell-based coadds and `metadetection`. Many parts are from the [Shear profile around A360 using ComCam HSM shapes](https://github.com/lsst-sitcom/comcam_clusters/blob/main/ACO360_WL_HSCcalib_CLMM.ipynb) notebook, especially the identification of red sequence galaxies and use of CLMM to create the tangential shear plot.

Last working weekly: `w_2025_28`

Container Size: 16 GB (large)

# Preparing data

## Imports & Definitions

In [ ]:
# locally install modeling packages (only do once, if not already installed)
# pip install pyccl
# pip install clmm

In [ ]:
from lsst.daf.butler import Butler

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib

import astropy.units as u
from astropy.table import Table
from astropy.coordinates import SkyCoord

import gc

from lsst.skymap import Index2D
import lsst.afw.geom as afwGeom
import lsst.afw.math as afwMath
import lsst.afw.image as afwImage
import lsst.geom as geom
import lsst.afw.display.rgb as afwRgb

# for stats control
from lsst.drp.tasks.assemble_cell_coadd import AssembleCellCoaddTask
import lsst.meas.algorithms as meas

from lsst.afw.geom.ellipses import Quadrupole, SeparableDistortionTraceRadius

from numpy.linalg import inv
import scipy.stats as stats

import clmm
from clmm import GalaxyCluster, ClusterEnsemble, GCData, Cosmology
from clmm import Cosmology, utils

cosmo = clmm.Cosmology(H0=70.0, Omega_dm0=0.3 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

%matplotlib inline

In [ ]:
REPO = '/sdf/data/rubin/repo/main/'
butler = Butler(REPO)
registry = butler.registry

collection = 'u/mgorsuch/metadetect/a360_3_band/20250707T015251Z' # with S/N fix
cell_collection = 'u/mgorsuch/a360_cell_coadd/20250513T044026Z' # cells used as input for r and i
cell_collection_g = 'u/mgorsuch/a360_cell_coadd_g/20250611T144112Z' # cells used as input for g

In [ ]:
# create and configure stats control object as seen in assemble_cell_coadd task
statsCtrl = afwMath.StatisticsControl()
statsCtrl.setAndMask(afwImage.Mask.getPlaneBitMask(("BAD", "NO_DATA", "SAT"))) # use default PlaneBitMasks from task
statsCtrl.setNanSafe(True)

## Read in data

Metadetect outputs tables for each patch. Read in each table and compile them together.

In [ ]:
# Position in degrees of the BCG for A360
ra_bcg = 37.865017
dec_bcg = 6.982205

The cell below is for finding the tracts/patches that are within the specified radius of the BCG. This is already incorporated in the butler collection used for the `metadetect` output.

In [ ]:
# skymap = butler.get('skyMap', skymap='lsst_cells_v1', collections='LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810')

# # Looking for all patches in delta deg region around it
# delta = 0.5
# center = geom.SpherePoint(ra_bcg, dec_bcg, geom.degrees)
# ra_min, ra_max = ra_bcg - delta, ra_bcg + delta
# dec_min, dec_max = dec_bcg - delta, dec_bcg + delta

# ra_range = (ra_min, ra_max)
# dec_range = (dec_min, dec_max)
# radec = [geom.SpherePoint(ra_range[0], dec_range[0], geom.degrees),
#          geom.SpherePoint(ra_range[0], dec_range[1], geom.degrees),
#          geom.SpherePoint(ra_range[1], dec_range[0], geom.degrees),
#          geom.SpherePoint(ra_range[1], dec_range[1], geom.degrees)]

# tractPatchList = skymap.findTractPatchList(radec)

# find dataset refs that are within the tract/patch list above
# datasetRefs_shear = []
# datasetRefs_catalog = []

# for tractPatch in tractPatchList:
#     tract = tractPatch[0]
#     patchInfo = tractPatch[1]
#     for patch in patchInfo:
#         datasetRefs_shear.append(butler.query_datasets('ShearObject', 
#                                                  collections=collection,
#                                                  tract=tract.tract_id,
#                                                  patch=patch.sequential_index))
#         datasetRefs_catalog.append(butler.query_datasets('objectTable', 
#                                                  collections=default_collection,
#                                                  tract=tract.tract_id,
#                                                  patch=patch.sequential_index))

In [ ]:
datasetRefs_shear = []
overlap_patches_10463 = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
tract_patch_list = [] # only used for plotting distribution of input images

for ref in butler.registry.queryDatasets('ShearObject', collections=collection):

    # the only parts of these tracts within 0.5 radius overlap with already included patches
    if ref.dataId['tract'] == 10704 or ref.dataId['tract'] == 10705:
        continue

    # these column of patches overlap with patches already in tract 10464
    if ref.dataId['tract'] == 10463 and ref.dataId['patch'] in overlap_patches_10463:
        continue
    
    datasetRefs_shear.append(butler.query_datasets('ShearObject', 
                                                     collections=collection,
                                                     skymap = 'lsst_cells_v1',
                                                     tract=ref.dataId['tract'],
                                                     patch=ref.dataId['patch']))

    tract_patch_list.append([ref.dataId['tract'], ref.dataId['patch']])

Combine the data from each patch into a single table.

In [ ]:
shear_table_list = []

for i, ref in enumerate(datasetRefs_shear):
    shear_data_patch = butler.get(ref[0])
    shear_table_patch = shear_data_patch.to_pandas()
    shear_table_list.append(shear_table_patch)

shear_table = pd.concat(shear_table_list)

# remove unused tables to clear up memory
del shear_table_list
gc.collect()

### Remove duplicate objects from patch overlap

In [ ]:
# copy table prior to duplicate removal for a later validation plot
shear_table_dup = shear_table.copy()

In [ ]:
# remove objects in outer ring of cells in each patch since patch overlap is two cells
# TO-DO: exempt rows that don't overlap with other patches, e.g. patches on the edge of the field
shear_table = shear_table[shear_table['cell_x']!=0]
shear_table = shear_table[shear_table['cell_x']!=21]
shear_table = shear_table[shear_table['cell_y']!=0]
shear_table = shear_table[shear_table['cell_y']!=21]
print("Number of rows after removing most duplicate cells: ", len(shear_table))

# some additional tract/patch overlap appears to have a 4 cell overlap
filt1 = shear_table['tract'] == 10464
filt1 &= shear_table['patch_x'] == 9
filt1 &= shear_table['cell_x'] == 20
shear_table = shear_table[np.invert(filt1)]

filt2 = shear_table['tract'] == 10463
filt2 &= shear_table['patch_x'] == 1
filt2 &= shear_table['cell_x'] == 1
shear_table = shear_table[np.invert(filt2)]
print("Number of rows after removing patch overlap in 10464: ", len(shear_table))

In [ ]:
# remove overlapping rows due to patch overlap    
print("Number of rows prior to removing duplicates: ", len(shear_table))
shear_table = shear_table.drop_duplicates(subset=['shear_type', 'ra', 'dec']) # each object will potentially have several sheared images
print("Number of rows after removing duplicates: ", len(shear_table))

### Add useful columns

In [ ]:
# make new columns to convert nJy fluxes to AB magnitudes
t1 = Table.from_pandas(shear_table)

t1['wmom_band_mag_g'] = (t1['wmom_band_flux_g']*u.nJy).to(u.ABmag)
t1['wmom_band_mag_r'] = (t1['wmom_band_flux_r']*u.nJy).to(u.ABmag)
t1['wmom_band_mag_i'] = (t1['wmom_band_flux_i']*u.nJy).to(u.ABmag)
t1['wmom_color_mag_g-r'] = t1['wmom_band_mag_g']-t1['wmom_band_mag_r']
t1['wmom_color_mag_g-i'] = t1['wmom_band_mag_g']-t1['wmom_band_mag_i']
t1['wmom_color_mag_r-i'] = t1['wmom_band_mag_r']-t1['wmom_band_mag_i']

shear_table = t1.to_pandas()

# Add columns for distance from BCG
c1 = SkyCoord(shear_table['ra'].values*u.deg, shear_table['dec'].values*u.deg)
c2 = SkyCoord(ra_bcg*u.deg, dec_bcg*u.deg)
sep = c1.separation(c2)

shear_table['deg_sep'] = sep.value

shear_table['mpc_sep'] = cosmo.eval_da(0.22) * shear_table['deg_sep'] * np.pi/180

# polar angle of source galaxy relative to BCG, from -pi to pi
shear_table['phi'] = np.arctan2(shear_table['dec'] - dec_bcg, (ra_bcg - shear_table['ra'])*np.cos(np.deg2rad(dec_bcg)))

# calculate tangential / cross shear components for each galaxy
# note that these shears need to be selected for ONLY the 'ns' (no artificial shear type)
shear_table['shear_t'] = -shear_table['wmom_g_1'] * np.cos(2*shear_table['phi']) \
                                - shear_table['wmom_g_2'] * np.sin(2*shear_table['phi'])
shear_table['shear_x'] = shear_table['wmom_g_1'] * np.sin(2*shear_table['phi']) \
                                - shear_table['wmom_g_2'] * np.cos(2*shear_table['phi'])

## Apply `metadetect` flags

Anything that is flagged should be removed.

In [ ]:
meta_filter = shear_table['wmom_flags']==False
meta_filter &= shear_table['psfrec_flags']==False
meta_filter &= shear_table['wmom_psf_flags']==False
meta_filter &= shear_table['wmom_obj_flags']==False
meta_filter &= shear_table['wmom_T_flags']==False
meta_filter &= shear_table['wmom_band_flux_flags_r']==False
meta_filter &= shear_table['wmom_band_flux_flags_i']==False

shear_table = shear_table[meta_filter]

print("Number of rows after removing metadetect flags: ", len(shear_table))
print("Number of rows in ns after removing metadetect flags: ", len(shear_table[shear_table['shear_type']=='ns']))

shear_table_md_cuts = shear_table.copy()

## Identify and remove cluster member galaxies

In [ ]:
# RS filter parameters
point_size = 0.8
point_alpha = 0.7

r_left = 19
r_right = 23
r_down_left = 0.45
r_up_left = 0.7
r_down_right = 0.3
r_up_right = 0.55
r_slope = (r_down_right - r_down_left) / (r_right - r_left)

g_left = 20
g_right = 23
g_down_left = 1.8
g_up_left = 2.1
g_down_right = 1.5
g_up_right = 1.8
g_slope = (g_down_right - g_down_left) / (g_right - g_left)

brightness_cut = 23

### RS Filter (one cell)

For applying RS cuts without running all of the visual inspection cells (this cell is based on those visual inspections).

In [ ]:
shear_table_wl = shear_table[shear_table['deg_sep'] < 0.5] # catalog for WL measurements
print("Number of rows after applying < 0.5 deg from center: ", len(shear_table_wl))

rs_hi_ri = r_up_left + r_slope * (shear_table_wl['wmom_band_mag_r']-r_left)
rs_low_ri = r_down_left + r_slope * (shear_table_wl['wmom_band_mag_r']-r_left)
ri_filt = shear_table_wl['wmom_color_mag_r-i'] > rs_low_ri
ri_filt &= shear_table_wl['wmom_color_mag_r-i'] < rs_hi_ri
ri_filt &= shear_table_wl['wmom_band_mag_r'] < brightness_cut_r

rs_hi_gi = g_up_left + g_slope * (shear_table_wl['wmom_band_mag_g']-g_left)
rs_low_gi = g_down_left + g_slope * (shear_table_wl['wmom_band_mag_g']-g_left)
gi_filt = shear_table_wl['wmom_color_mag_g-i'] > rs_low_gi
gi_filt &= shear_table_wl['wmom_color_mag_g-i'] < rs_hi_gi
gi_filt &= shear_table_wl['wmom_band_mag_g'] < brightness_cut_g

rs_filt = np.logical_or(ri_filt, gi_filt)
RS_id_list = shear_table_wl['id'][rs_filt]

# Filter out rows where the 'dataid' column matches any value in RS_id_list
shear_table_wl = shear_table_wl[~shear_table_wl['id'].isin(RS_id_list)]
shear_table_wl_ns = shear_table_wl[shear_table_wl['shear_type']=='ns']

print("Number of rows after 0.5 degree cut and RS cuts: ", len(shear_table_wl))
print("Number of rows after 0.5 degree cut and RS cuts, ns only: ", len(shear_table_wl_ns))

shear_table_rs_cuts = shear_table_wl.copy()

### Visual Inspections

Cuts are applied to all shear type catalogs, though only non-sheared are plotted. This section doesn't need to be run if the above cell was already run.

In [ ]:
# isolate no shear catalog
shear_table_ns = shear_table[shear_table['shear_type']=='ns']

# distribution of magnitudes prior to cuts
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12,4))
ax[0].hist(shear_table_ns['wmom_band_mag_g'], bins=75)
ax[0].set_title("g-band")
ax[0].vlines(x=25.7, ymin=0, ymax=50000, color='red')
ax[1].hist(shear_table_ns['wmom_band_mag_r'], bins=75)
ax[1].set_title("r-band")
ax[1].vlines(x=25.4, ymin=0, ymax=50000, color='red')
ax[2].hist(shear_table_ns['wmom_band_mag_i'], bins=75)
ax[2].set_title("i-band")
ax[2].vlines(x=25.1, ymin=0, ymax=50000, color='red')

for ax in ax.reshape(-1):
    ax.set_yscale('log')
    ax.set_xlabel("AB Mag")
    ax.set_xlim(right=29)
    ax.set_ylim(top=50000)

plt.savefig('image_outputs_fix/object-magnitudes.png', bbox_inches='tight')
plt.suptitle("Object Magnitudes")

#### Color-Magnitude Plots

In [ ]:
# Filter objects to within < 0.1 deg of cluster center
filt = shear_table['deg_sep'] < 0.1 # stay close to cluster center for RS indentification
shear_table_rs = shear_table[shear_table['deg_sep'] < 0.1] # catalog for RS identification
print("Number of rows in after applying < 0.1 deg from center for all shear types: ", len(shear_table_rs))

In [ ]:
shear_table_rs_ns = shear_table_rs[shear_table_rs['shear_type']=='ns']

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_rs_ns['wmom_band_mag_r'], shear_table_rs_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size, alpha=point_alpha)
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].plot([r_left,r_right],[r_down_left,r_down_right], color='r', linewidth=0.7)
axes[0].plot([r_left,r_right],[r_up_left,r_up_right], color='r', linewidth=0.7)
axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,27])

axes[1].scatter(shear_table_rs_ns['wmom_band_mag_g'], shear_table_rs_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size, alpha=point_alpha)
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].plot([g_left,g_right],[g_down_left,g_down_right], color='r', linewidth=0.7)
axes[1].plot([g_left,g_right],[g_up_left,g_up_right], color='r', linewidth=0.7)
axes[1].set_ylim([-0.5,3])
axes[1].set_xlim([18,27])

plt.suptitle("Color-Magnitude < 0.1 deg")

plt.savefig('image_outputs_fix/color-magnitude-0-1.png', bbox_inches='tight')
plt.show()

Note that combining all catalogs causes a "smeary" look is due to the 5 sheared/unsheared images of the same object that are detected & measured in slightly different ways.

#### Filter RS galaxies

In [ ]:
rs_hi_ri = r_up_left + r_slope * (shear_table_rs['wmom_band_mag_r']-r_left)
rs_low_ri = r_down_left + r_slope * (shear_table_rs['wmom_band_mag_r']-r_left)
ri_filt = shear_table_rs['wmom_color_mag_r-i'] > rs_low_ri
ri_filt &= shear_table_rs['wmom_color_mag_r-i'] < rs_hi_ri
ri_filt &= shear_table_rs['wmom_band_mag_r'] < brightness_cut

rs_hi_gi = g_up_left + g_slope * (shear_table_rs['wmom_band_mag_g']-g_left)
rs_low_gi = g_down_left + g_slope * (shear_table_rs['wmom_band_mag_g']-g_left)
gi_filt = shear_table_rs['wmom_color_mag_g-i'] > rs_low_gi
gi_filt &= shear_table_rs['wmom_color_mag_g-i'] < rs_hi_gi
gi_filt &= shear_table_rs['wmom_band_mag_g'] < brightness_cut

# ns only, for plotting
ri_filt_ns = np.logical_and(ri_filt, shear_table_rs['shear_type'] == 'ns')
gi_filt_ns = np.logical_and(gi_filt, shear_table_rs['shear_type'] == 'ns')

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_rs_ns['wmom_band_mag_r'], shear_table_rs_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size) # all galaxies  
axes[0].scatter(shear_table_rs['wmom_band_mag_r'][ri_filt_ns], 
           shear_table_rs['wmom_color_mag_r-i'][ri_filt_ns], 
           marker='.', s=point_size) #red sequence galaxies
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].plot([r_left,r_right],[r_down_left,r_down_right], color='r', linewidth=0.7)
axes[0].plot([r_left,r_right],[r_up_left,r_up_right], color='r', linewidth=0.7)
axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,27])

axes[1].scatter(shear_table_rs_ns['wmom_band_mag_g'], shear_table_rs_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size) # all galaxies  
axes[1].scatter(shear_table_rs['wmom_band_mag_g'][gi_filt_ns], 
           shear_table_rs['wmom_color_mag_g-i'][gi_filt_ns], 
           marker='.', s=point_size)
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].plot([g_left,g_right],[g_down_left,g_down_right], color='r', linewidth=0.7)
axes[1].plot([g_left,g_right],[g_up_left,g_up_right], color='r', linewidth=0.7)
axes[1].set_ylim([-0.5,3])
axes[1].set_xlim([18,27])

plt.suptitle("Color-Magnitude < 0.1 deg\nCuts highlighted")

plt.savefig('image_outputs_fix/color-magnitude-0-1-orange.png', bbox_inches='tight')
plt.show()

Cut out the RS galaxies identified above.

In [ ]:
shear_table_wl = shear_table[shear_table['deg_sep'] < 0.5] # catalog for WL measurements
print("Number of rows in ns after applying < 0.5 deg from center: ", len(shear_table_wl))

In [ ]:
rs_hi_ri = r_up_left + r_slope * (shear_table_wl['wmom_band_mag_r']-r_left)
rs_low_ri = r_down_left + r_slope * (shear_table_wl['wmom_band_mag_r']-r_left)
ri_filt = shear_table_wl['wmom_color_mag_r-i'] > rs_low_ri
ri_filt &= shear_table_wl['wmom_color_mag_r-i'] < rs_hi_ri
ri_filt &= shear_table_wl['wmom_band_mag_r'] < brightness_cut

rs_hi_gi = g_up_left + g_slope * (shear_table_wl['wmom_band_mag_g']-g_left)
rs_low_gi = g_down_left + g_slope * (shear_table_wl['wmom_band_mag_g']-g_left)
gi_filt = shear_table_wl['wmom_color_mag_g-i'] > rs_low_gi
gi_filt &= shear_table_wl['wmom_color_mag_g-i'] < rs_hi_gi
gi_filt &= shear_table_wl['wmom_band_mag_g'] < brightness_cut

# ns only, for plotting
ri_filt_ns = np.logical_and(ri_filt, shear_table_wl['shear_type'] == 'ns')
gi_filt_ns = np.logical_and(gi_filt, shear_table_wl['shear_type'] == 'ns')

shear_table_wl_ns = shear_table_wl[shear_table_wl['shear_type'] == 'ns']

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_wl_ns['wmom_band_mag_r'], shear_table_wl_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size) # all galaxies  
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,27])

axes[1].scatter(shear_table_wl_ns['wmom_band_mag_g'], shear_table_wl_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size) 
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].set_ylim([-0.5,3])
axes[1].set_xlim([18,27])

plt.suptitle("Color-Magnitude < 0.5 deg")

plt.savefig('image_outputs_fix/color-magnitude-0-5-no-line.png', bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_wl_ns['wmom_band_mag_r'], shear_table_wl_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size) # all galaxies  
axes[0].scatter(shear_table_wl['wmom_band_mag_r'][ri_filt_ns], 
           shear_table_wl['wmom_color_mag_r-i'][ri_filt_ns], 
           marker='.', s=point_size) #red sequence galaxies
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].plot([r_left,r_right],[r_down_left,r_down_right], color='r', linewidth=0.7)
axes[0].plot([r_left,r_right],[r_up_left,r_up_right], color='r', linewidth=0.7)
axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,27])

axes[1].scatter(shear_table_wl_ns['wmom_band_mag_g'], shear_table_wl_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size) # all galaxies  
axes[1].scatter(shear_table_wl['wmom_band_mag_g'][gi_filt_ns], 
           shear_table_wl['wmom_color_mag_g-i'][gi_filt_ns], 
           marker='.', s=point_size)
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].plot([g_left,g_right],[g_down_left,g_down_right], color='r', linewidth=0.7)
axes[1].plot([g_left,g_right],[g_up_left,g_up_right], color='r', linewidth=0.7)
axes[1].set_ylim([-0.5,3])
axes[1].set_xlim([18,27])

plt.suptitle("Color-Magnitude < 0.5 deg\nCuts highlighted")

plt.savefig('image_outputs_fix/color-magnitude-0-5-orange.png', bbox_inches='tight')
plt.show()

In [ ]:
# rs_filt = np.logical_and(ri_filt, gi_filt)
rs_filt = np.logical_or(ri_filt, gi_filt)
rs_filt_ns = np.logical_and(rs_filt, shear_table_wl['shear_type'] == 'ns')

RS_id_list = shear_table_wl['id'][rs_filt]
RS_id_list_ns = shear_table_wl_ns['id'][rs_filt_ns] # for plotting

In [ ]:
print(len(RS_id_list))
print(len(RS_id_list_ns)) # should be ~20% of above line

In [ ]:
# Filter out rows where the 'dataid' column matches any value in RS_id_list
shear_table_wl = shear_table_wl[~shear_table_wl['id'].isin(RS_id_list)]
shear_table_wl_ns = shear_table_wl_ns[~shear_table_wl_ns['id'].isin(RS_id_list_ns)]

In [ ]:
print(len(shear_table_wl))
print(len(shear_table_wl_ns))

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_wl_ns['wmom_band_mag_r'], shear_table_wl_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size) # all galaxies  
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,27])

axes[1].scatter(shear_table_wl_ns['wmom_band_mag_g'], shear_table_wl_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size) 
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].set_ylim([-0.5,3])
axes[1].set_xlim([18,27])

plt.suptitle("Color-Magnitude < 0.5 deg\nCuts removed")

plt.savefig('image_outputs_fix/color-magnitude-0-5-removed.png', bbox_inches='tight')
plt.show()

### Check lines of overdensity

Lines of overdensity: imperfect overlap of objects between patches/tracts.

(Outdated, but code might be useful later)

In [ ]:
# patch_list = []

# for ref in butler.registry.queryDatasets('deepCoaddCell', collections=cell_collection, band='i'):
#     if ref.dataId['tract'] == 10704 or ref.dataId['tract'] == 10705:
#         continue

#     if ref.dataId['tract'] == 10463 and ref.dataId['patch'] in overlap_patches_10463:
#         continue
    
#     patch_list.append(butler.query_datasets('deepCoaddCell', 
#                                                  collections=cell_collection,
#                                                  skymap = 'lsst_cells_v1',
#                                                  band = 'i',
#                                                  tract=ref.dataId['tract'],
#                                                  patch=ref.dataId['patch'])[0])

In [ ]:
# segs = []

# for ref in patch_list:
    
#     coadd = butler.get('deepCoaddCell',
#                       collections=cell_collection,
#                       skymap = 'lsst_cells_v1',
#                       band = 'i',
#                       tract=ref.dataId['tract'],
#                       patch=ref.dataId['patch'])

#     wcs = coadd.wcs
#     bbox = coadd.inner_bbox

#     coadd_corners = coadd.inner_bbox.getCorners()

#     for index, corner in enumerate(coadd_corners):
#         corner_coord_start = wcs.pixelToSky(corner.getX(), corner.getY())
#         if index < 3:
#             corner_coord_end = wcs.pixelToSky(coadd_corners[index+1].getX(), coadd_corners[index+1].getY())
#         else:
#             corner_coord_end = wcs.pixelToSky(coadd_corners[0].getX(), coadd_corners[0].getY())
    
#         start_ra = corner_coord_start[0].asDegrees()
#         start_dec = corner_coord_start[1].asDegrees()
    
#         end_ra = corner_coord_end[0].asDegrees()
#         end_dec = corner_coord_end[1].asDegrees()

#         segs.append(((start_ra, start_dec), (end_ra, end_dec)))

#     del coadd
#     gc.collect()

In [ ]:
# plt.scatter(ra, dec, marker='.', s=0.2)
# plt.scatter([ra_bcg], [dec_bcg], marker='+', s=100, color='orange')
# for seg in segs:
#     plt.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.4)
# plt.show()

## Quality cuts

Using cuts based on [Yamamoto 2024](https://arxiv.org/abs/2501.05665), though some changes have been made to account for LSSTComCam specific data.

In [ ]:
def print_rows_removed(og_table, cuts_name):
    total = len(og_table)
    print("Cut name: ", cuts_name)
    if cuts_name=="final_cuts":
        print("wmom_T_ratio rows removed: ", total-len(og_table[og_table['wmom_T_ratio']>1.1]), "  (", \
            (1 - len(og_table[og_table['wmom_T_ratio']>1.1]) / total))
        print("wmom_s2n rows removed: ", total-len(og_table[og_table['wmom_s2n']>10]), "  (", \
            (1 - len(og_table[og_table['wmom_s2n']>10]) / total))
        print("wmom_T rows removed: ", total-len(og_table[og_table['wmom_T']<20]), "  (", \
            (1 - len(og_table[og_table['wmom_T']<20]) / total))
        print("m_frac rows removed: ", total-len(og_table[og_table['mfrac']<0.1]), "  (", \
            (1 - len(og_table[og_table['mfrac']<0.1]) / total))
        print("wmom_band_mag_g rows removed: ", total-len(og_table[og_table['wmom_band_mag_g']<25.7]), "  (", \
            (1 - len(og_table[og_table['wmom_band_mag_g']<25.7]) / total))
        print("wmom_band_mag_r rows removed: ", total-len(og_table[og_table['wmom_band_mag_r']<25.4]), "  (", \
            (1 - len(og_table[og_table['wmom_band_mag_r']<25.4]) / total))
        print("wmom_band_mag_i rows removed: ", total-len(og_table[og_table['wmom_band_mag_i']<25.1]), "  (", \
            (1 - len(og_table[og_table['wmom_band_mag_i']<25.1]) / total))
        print("wmom_color_mag_g-r rows removed: ", total-len(og_table[(og_table['wmom_color_mag_g-r']).abs()<5]), "  (", \
            (1 - len(og_table[(og_table['wmom_color_mag_g-r']).abs()<5]) / total))
        print("wmom_color_mag_r-i rows removed: ", total-len(og_table[(og_table['wmom_color_mag_r-i']).abs()<5]), "  (", \
            (1 - len(og_table[(og_table['wmom_color_mag_r-i']).abs()<5]) / total))
        print("wmom_band_mag_i<20 rows removed: ", total-len(og_table[og_table['wmom_band_mag_i']>20]), "  (", \
            (1 - len(og_table[og_table['wmom_band_mag_i']>20]) / total))
        print("Junk cut 1: ", total-len(og_table[og_table['wmom_T'] < (0.41 - 1.4*og_table['wmom_T_err'])]), "  (", \
            (1 - len(og_table[og_table['wmom_T'] < (0.41 - 1.4*og_table['wmom_T_err'])]) / total))
        print("Junk cut 2: ", total-len(og_table[(og_table['wmom_T'] * og_table['wmom_T_err']) < 0.015]), "  (", \
            (1 - len(og_table[(og_table['wmom_T'] * og_table['wmom_T_err']) < 0.015]) / total))
    else:
        print("Not a valid cut or missing column name")

In [ ]:
# cuts determined mainly from Yamamoto but also additional analyses here
final_cuts = shear_table_wl['wmom_T_ratio']>1.1
final_cuts &= shear_table_wl['wmom_s2n']>10
final_cuts &= shear_table_wl['wmom_T']<20
final_cuts &= shear_table_wl['mfrac']<0.1
final_cuts &= shear_table_wl['wmom_band_mag_g']<25.7
final_cuts &= shear_table_wl['wmom_band_mag_r']<25.4
final_cuts &= shear_table_wl['wmom_band_mag_i']<25.1
final_cuts &= (shear_table_wl['wmom_color_mag_g-r']).abs()<5
final_cuts &= (shear_table_wl['wmom_color_mag_r-i']).abs()<5
final_cuts &= shear_table_wl['wmom_band_mag_i']>20
# junk cuts
final_cuts &= shear_table_wl['wmom_T'] < (0.41 - 1.4*shear_table_wl['wmom_T_err'])
# final_cuts &= (shear_table_wl['wmom_T'] * shear_table_wl['wmom_T_err']) < 0.015

print(len(shear_table_wl))
cut_name, cut_name_string = final_cuts, "final_cuts"
print_rows_removed(shear_table_wl, cut_name_string)

shear_table_wl = shear_table_wl[cut_name]
shear_table_wl_ns = shear_table_wl[shear_table_wl['shear_type']=='ns']

print()
print("Number of rows after applying quality cuts: ", len(shear_table_wl))
print("Number of rows in ns after applying quality cuts: ", len(shear_table_wl_ns))

# Looking at shear outputs

## Check shear types for each object

Each object is detected and measured separately for each sheared/unsheared image. The catalogs will not necessarily be the same but should be close in number of objects.

In [ ]:
# split catalog by shear type
shear_table_wl_ns = shear_table_wl[shear_table_wl['shear_type']=='ns']
shear_table_wl_1p = shear_table_wl[shear_table_wl['shear_type']=='1p']
shear_table_wl_1m = shear_table_wl[shear_table_wl['shear_type']=='1m']
shear_table_wl_2p = shear_table_wl[shear_table_wl['shear_type']=='2p']
shear_table_wl_2m = shear_table_wl[shear_table_wl['shear_type']=='2m']

In [ ]:
print("Number of shear type 'ns': ", len(shear_table_wl_ns))
print("Number of shear type '1p': ", len(shear_table_wl_1p))
print("Number of shear type '1m': ", len(shear_table_wl_1m))
print("Number of shear type '2p': ", len(shear_table_wl_2p))
print("Number of shear type '2m': ", len(shear_table_wl_2m))

## Determining tangential & cross shear

In [ ]:
# Radial binning, either in Mpc or degrees
bins_deg = clmm.make_bins(0.025,0.5,nbins=6, method='evenlog10width')
bins_mpc = cosmo.eval_da(0.22) * bins_deg * np.pi/180

# define distance bins
dig_rad_bins_ns = np.digitize(shear_table_wl_ns['deg_sep'], bins_deg)
dig_mpc_bins_ns = np.digitize(shear_table_wl_ns['mpc_sep'], bins_mpc)

shear_diff = 0.02

Definitions of tangential and cross shear used:
$$ \gamma_t = -\gamma_1\cos(2\phi)-\gamma_2\sin(2\phi) $$
$$ \gamma_\times = \gamma_1\sin(2\phi)-\gamma_2\cos(2\phi) $$

$\phi$ : Polar Angle of galaxy relative to BCG 

### Individual Phi

This section will attempt to measure $\gamma_t$ and $\gamma_\times$ by:
- Calculate the response R from all galaxies, no binning
- Apply R to mean $\gamma_t,\gamma_\times$ of each bin, getting the calibrated shear

In [ ]:
p1_mean = shear_table_wl_1p['wmom_g_1'].mean()
m1_mean = shear_table_wl_1m['wmom_g_1'].mean()
p2_mean = shear_table_wl_2p['wmom_g_2'].mean()
m2_mean = shear_table_wl_2m['wmom_g_2'].mean()

r_matrix = [[0, 0],[0, 0]]

r_matrix[0][0] = (p1_mean - m1_mean) / shear_diff
# r_matrix[0][1] = (p2_mean - m2_mean) / shear_diff
# r_matrix[1][0] = (p1_mean - m1_mean) / shear_diff
r_matrix[1][1] = (p2_mean - m2_mean) / shear_diff

r_matrix_inv = inv(r_matrix)

# calculate R error
p1_err = np.std(shear_table_wl_1p['wmom_g_1']) / np.sqrt(len(shear_table_wl_1p['wmom_g_1']))
m1_err = np.std(shear_table_wl_1m['wmom_g_1']) / np.sqrt(len(shear_table_wl_1m['wmom_g_1']))
p2_err = np.std(shear_table_wl_2p['wmom_g_2']) / np.sqrt(len(shear_table_wl_2p['wmom_g_2']))
m2_err = np.std(shear_table_wl_2m['wmom_g_2']) / np.sqrt(len(shear_table_wl_2m['wmom_g_2']))

r11_err = np.sqrt(p1_err**2 + m1_err**2)
r22_err = np.sqrt(p2_err**2 + m2_err**2)

print("R11, R22: ", r_matrix[0][0], r_matrix[1][1])
print("R11_err, R22_err: ", r11_err, r22_err)
print("Difference of R11 and R22: ", np.abs(r_matrix[0][0]-r_matrix[1][1]))

In [ ]:
# bin tangential and cross shears by radial bins
tan_cross_shears = np.zeros((len(bins_mpc)-1, 2)) # binned tangential and cross shear
mean_mpc = []
tan_errs = []
cross_errs = []

for i in range(0, len(bins_mpc)-1):
    bin_filt_ns = dig_mpc_bins_ns == i+1

    # print number of galaxies in each bin
    print("Rows in bin ", i, " :", len(shear_table_wl_ns['shear_t'][bin_filt_ns]))

    # calulcate mean t and x shears
    mean_g_t = shear_table_wl_ns['shear_t'][bin_filt_ns].mean() # mean g1
    mean_g_x = shear_table_wl_ns['shear_x'][bin_filt_ns].mean() # mean g2

    # calculate errors, 95% confidence interval
    g_t_err = stats.bootstrap([shear_table_wl_ns['shear_t'][bin_filt_ns]], np.mean).confidence_interval
    g_x_err = stats.bootstrap([shear_table_wl_ns['shear_x'][bin_filt_ns]], np.mean).confidence_interval

    # apply calibration 
    shear_cal = r_matrix_inv.dot([mean_g_t, mean_g_x])
    shear_cal_err_upper = r_matrix_inv.dot([g_t_err.high, g_x_err.high])
    shear_cal_err_lower = r_matrix_inv.dot([g_t_err.low, g_x_err.low])

    tan_err = [shear_cal_err_lower[0], shear_cal_err_upper[0]]
    cross_err = [shear_cal_err_lower[1], shear_cal_err_upper[1]]

    tan_cross_shears[i] = shear_cal
    tan_errs.append(tan_err)
    cross_errs.append(cross_err)

    # get the mean distance from BCG
    mean_deg_sep = shear_table_wl_ns['deg_sep'][bin_filt_ns].mean()
    mean_mpc_sep = cosmo.eval_da(0.22) * mean_deg_sep * np.pi/180
    mean_mpc.append(mean_mpc_sep)

tan_errs = np.array(tan_errs)
cross_errs = np.array(cross_errs)

#### Getting NFW Model with CLMM Package

In [ ]:
galcat = GCData()
galcat['ra'] = shear_table_wl_ns['ra']
galcat['dec'] = shear_table_wl_ns['dec']
galcat['e1'] = shear_table_wl_ns['wmom_g_1'] * 2
galcat['e2'] = shear_table_wl_ns['wmom_g_2'] * 2

galcat['z'] = np.zeros(len(shear_table_wl_ns)) # CLMM needs a redshift column for the source, even if not used

cluster_id = "Abell 360"
gc_object1 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, coordinate_system='euclidean')

gc_object1.compute_galaxy_weights(
        shape_component1="e1",
        shape_component2="e2",
        use_shape_error=False, # individual shape errors are not yet available for Metadetect
        use_shape_noise=True,
        weight_name="w_ls",
        cosmo=cosmo,
        add=True,
    )

moo = clmm.Modeling(massdef="mean", delta_mdef=500, halo_profile_model="nfw")

moo.set_cosmo(cosmo)
moo.set_concentration(4)
# moo.set_mass(1.0e15)
moo.set_mass(4.0e14)

z_cl = gc_object1.z

# source properties
# assume sources redshift following a the DESC SRD distribution. This will need updating.

z_distrib_func = utils.redshift_distributions.desc_srd

# Compute first beta (e.g. eq(6) of WtGIII paper)
beta_kwargs = {
    "z_cl": z_cl,
    "z_inf": 10.0,
    "cosmo": cosmo,
    "z_distrib_func": z_distrib_func,
}
beta_s_mean = utils.compute_beta_s_mean_from_distribution(**beta_kwargs)
beta_s_square_mean = utils.compute_beta_s_square_mean_from_distribution(**beta_kwargs)

rproj = np.logspace(np.log10(0.3),np.log10(7.), 100)

gt_z = moo.eval_reduced_tangential_shear(
    rproj, z_cl, [beta_s_mean, beta_s_square_mean], z_src_info="beta", approx="order2"
)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(8,6))

point_size = 40
point_alpha = 1

# g1 calibrated
axes.scatter(mean_mpc, tan_cross_shears[:,0],
             marker='.', s=point_size, label='tangential shear')
axes.plot(mean_mpc, tan_cross_shears[:,0], '-o')
axes.vlines(mean_mpc, tan_errs[:,0], tan_errs[:,1], colors='blue')

cross_axis = np.add(mean_mpc, 0.06) # add offset to visually differentiate

axes.scatter(cross_axis, tan_cross_shears[:,1],
             marker='.', s=point_size, label='cross shear')
axes.plot(cross_axis, tan_cross_shears[:,1], '-o')
axes.vlines(cross_axis, cross_errs[:,0], cross_errs[:,1], colors='orange')

plt.axhline(0.0, color='k', ls=':')

plt.plot(rproj, gt_z, label='NFW (model, not fit), M500c=4e14 Msun, c=4, n(z)=SRD', ls=':')

plt.xscale('log')
plt.axhline(0.0, color='k', ls=':')
plt.ylim([-0.06,0.1])
plt.xlim([0.3,7])
axes.set_xlabel('R [Mpc]')
axes.set_ylabel("reduced shear")
axes.legend(loc=1)

plt.suptitle("")
plt.savefig('image_outputs_fix/shear-final.png', bbox_inches='tight')
plt.show()

# Validation

## Check g1/g2 and PSF ellipticities

Check that the PSF and object ellipticities average to ~0

In [ ]:
# print mean values
print("Mean of psfrec_g_1: ", shear_table_wl_ns['psfrec_g_1'].median())
print("Mean of psfrec_g_2: ", shear_table_wl_ns['psfrec_g_2'].median())
print("Mean of wmom_g_1: ", shear_table_wl_ns['wmom_g_1'].median())
print("Mean of wmom_g_2: ", shear_table_wl_ns['wmom_g_2'].median())

In [ ]:
n_bins = 50

fig, axs = plt.subplots(1, 4, sharey=True, tight_layout=True)

axs[0].hist(shear_table_wl_ns['psfrec_g_1'], bins=n_bins)
axs[0].set_title('psfrec_g_1')
axs[1].hist(shear_table_wl_ns['psfrec_g_2'], bins=n_bins)
axs[1].set_title('psfrec_g_2')
axs[2].hist(shear_table_wl_ns['wmom_g_1'], bins=n_bins)
axs[2].set_title('wmom_g_1')
axs[3].hist(shear_table_wl_ns['wmom_g_2'], bins=n_bins)
axs[3].set_title('wmom_g_2')

fig.suptitle("Distribution of ellipticities for PSF and WMOM")

plt.show()

In [ ]:
n_bins = 50

fig, axs = plt.subplots(1, 4, sharey=True, tight_layout=True)

axs[0].hist(shear_table_wl_ns['psfrec_g_1'], bins=n_bins)
axs[0].set_yscale('log')
axs[0].set_title('psfrec_g_1')
axs[1].hist(shear_table_wl_ns['psfrec_g_2'], bins=n_bins)
axs[1].set_title('psfrec_g_2')
axs[1].set_yscale('log')
axs[2].hist(shear_table_wl_ns['wmom_g_1'], bins=n_bins)
axs[2].set_title('wmom_g_1')
axs[2].set_yscale('log')
axs[3].hist(shear_table_wl_ns['wmom_g_2'], bins=n_bins)
axs[3].set_title('wmom_g_2')
axs[3].set_yscale('log')

fig.suptitle("Distribution of ellipticities for PSF and WMOM")

plt.show()

## Metadetect Limiting Magnitude vs. `cmodel` Limiting Magnitude

In [ ]:
# get the cmodel magnitudes

c_repo = '/repo/dp1'
c_collection = 'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260'
c_butler = Butler(c_repo, collections=c_collection)

obj_table_list = []

for tract, patch in tract_patch_list:
    
    dataId = {'tract': tract, 'patch' : patch , 'skymap':'lsst_cells_v1'}
    obj_cat = c_butler.get('object_patch', dataId=dataId, collections=c_collection)
    obj_table = obj_cat.to_pandas()

    # apply cuts from CLMM/HSM notebook, though adding g-band
    filt = obj_cat['detect_isPrimary']==True
    filt &= obj_cat['g_cModel_flag']== False
    filt &= obj_cat['r_cModel_flag']== False
    filt &= obj_cat['i_cModel_flag']== False
    filt &= obj_cat['g_cModelFlux']>0
    filt &= obj_cat['r_cModelFlux']>0
    filt &= obj_cat['i_cModelFlux']>0
    filt &= obj_cat['refExtendedness'] > 0.5

    # grab only cmodel fluxes to keep memory usage down
    obj_table = obj_table[['g_cModelFlux', 'r_cModelFlux', 'i_cModelFlux']]

    obj_table_list.append(obj_table[filt])

cmodel_table = pd.concat(obj_table_list)

# remove unused tables to clear up memory
del obj_table_list
gc.collect()

In [ ]:
# convert fluxes to magnitudes
cmodel_mag_g = -2.50 * np.log10(cmodel_table['i_cModelFlux']) + 31.4
cmodel_mag_r = -2.50 * np.log10(cmodel_table['i_cModelFlux']) + 31.4
cmodel_mag_i = -2.50 * np.log10(cmodel_table['r_cModelFlux']) + 31.4

In [ ]:
# isolate no shear catalog
shear_table_md_cuts_ns = shear_table_md_cuts[shear_table_md_cuts['shear_type']=='ns']

# distribution of magnitudes prior to cuts, Metadetection
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(15,10))
ax[0][0].hist(shear_table_md_cuts_ns['wmom_band_mag_g'], bins=75)
ax[0][0].set_title("g-band")
ax[0][0].vlines(x=25.7, ymin=0, ymax=35000, color='red')
ax[0][0].set_ylabel("Metadetection")
ax[0][1].hist(shear_table_md_cuts_ns['wmom_band_mag_r'], bins=75)
ax[0][1].set_title("r-band")
ax[0][1].vlines(x=25.4, ymin=0, ymax=35000, color='red')
ax[0][2].hist(shear_table_md_cuts_ns['wmom_band_mag_i'], bins=75)
ax[0][2].set_title("i-band")
ax[0][2].vlines(x=25.1, ymin=0, ymax=35000, color='red')

# distribution of magnitudes with cmodel
ax[1][0].hist(cmodel_mag_g, bins=75)
ax[1][0].set_title("g-band")
ax[1][0].vlines(x=25.7, ymin=0, ymax=30000, color='red')
ax[1][0].set_ylabel("cmodel")
ax[1][1].hist(cmodel_mag_r, bins=75)
ax[1][1].set_title("r-band")
ax[1][1].vlines(x=25.4, ymin=0, ymax=30000, color='red')
ax[1][2].hist(cmodel_mag_i, bins=75)
ax[1][2].set_title("i-band")
ax[1][2].vlines(x=25.1, ymin=0, ymax=30000, color='red')

for ax in ax.reshape(-1):
    ax.set_yscale('log')
    ax.set_xlabel("AB Mag")
    ax.set_xlim(right=32)
    #ax.set_ylim(top=14000)

plt.savefig('image_outputs_fix/object-magnitudes-compare.png', bbox_inches='tight')
plt.suptitle("Object Magnitudes\nMetadetection vs. cmodel")
plt.show()

## Plot `wmom_T_ratio` vs S/N

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(7,6))

shear_table_rs_cuts_ns = shear_table_rs_cuts[shear_table_rs_cuts['shear_type']=='ns']

axes.scatter(np.log10(shear_table_rs_cuts_ns['wmom_s2n']), shear_table_rs_cuts_ns['wmom_T_ratio'], 
           marker='.', s=3, alpha=point_alpha, label="All")
axes.scatter(np.log10(shear_table_wl_ns['wmom_s2n']), shear_table_wl_ns['wmom_T_ratio'], 
           marker='.', s=3, alpha=point_alpha, label='Remaining')
axes.set_xlabel('$log_{10}$(S/N)')
axes.set_ylabel('wmom_T_ratio')
axes.hlines(1.1, 0.5, 5, color='red', label='1.1 limit')
plt.legend()

plt.savefig('image_outputs_fix/obj_T_vs_s2n.png', bbox_inches='tight')
plt.show()

## Plot object density

In [ ]:
obj_density = shear_table_rs_cuts.copy()
obj_density = obj_density[obj_density['shear_type']=='ns']

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(19,6), sharex=True, sharey=True)
plt.subplots_adjust(wspace=0, hspace=0)

axes[0].scatter(shear_table_dup['ra'][shear_table_dup['shear_type']=='ns'], shear_table_dup['dec'][shear_table_dup['shear_type']=='ns'], marker='.', s=0.1, alpha=0.6) 
axes[0].set_title("No cuts")
axes[0].set_xlabel("ra")
axes[0].set_ylabel("dec")
axes[0].invert_xaxis()

axes[1].scatter(obj_density['ra'], obj_density['dec'], marker='.', s=0.1, alpha=0.6) 
axes[1].set_title("Duplicate and Red Sequence Cuts")
axes[1].set_xlabel("ra")
axes[1].invert_xaxis()

axes[2].scatter(shear_table_wl_ns['ra'], shear_table_wl_ns['dec'], marker='.', s=0.1, alpha=0.6) 
axes[2].set_title("All cuts")
axes[2].set_xlabel("ra")
axes[2].set_xlim(ra_bcg-0.7, ra_bcg+0.7)
axes[2].set_ylim(dec_bcg-0.7, dec_bcg+0.7)
axes[2].invert_xaxis()

for ax in axes.reshape(-1):
    ax.scatter([ra_bcg], [dec_bcg], marker='+', s=100, color='orange')

plt.suptitle("Object Distribution", size=16, y=0.98)
plt.savefig('image_outputs_fix/object-distribution-before-after.png', bbox_inches='tight')
plt.show()

## Junk Detections

In [ ]:
x_1 = np.linspace(0.0, 0.06, num=10)
line_y = 0.41 - 1.4*x_1

fig, axes = plt.subplots(1,1, figsize=(7,7), sharex=True, sharey=True)
plt.subplots_adjust(wspace=0, hspace=0)

axes.scatter(shear_table_rs_cuts['wmom_T_err'][shear_table_rs_cuts['shear_type']=='ns'], 
                shear_table_rs_cuts['wmom_T'][shear_table_rs_cuts['shear_type']=='ns'], marker='.', s=2, label='All')
axes.scatter(shear_table_wl['wmom_T_err'][shear_table_wl['shear_type']=='ns'], 
                shear_table_wl['wmom_T'][shear_table_wl['shear_type']=='ns'], marker='.', s=2, label='Remaining')
axes.plot(x_1, line_y, color='red', linewidth=0.7, label='T versus T_err cut')
axes.set_xlabel("wmom_T_err")
axes.set_ylabel("wmom_T")
plt.legend()

plt.savefig('image_outputs_fix/junk1.png', bbox_inches='tight')
plt.show()

In [ ]:
print((shear_table_rs_cuts['wmom_T']*shear_table_rs_cuts['wmom_T_err']).max())
print((shear_table_wl['wmom_T']*shear_table_wl['wmom_T_err']).max())

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(14,6), sharex=True, sharey=True)

axes[0].hist(shear_table_rs_cuts['wmom_T']*shear_table_rs_cuts['wmom_T_err'], bins=75)
axes[0].vlines(0.015, 0, 22000, color='red')
axes[0].set_title("T x T_err (after RS cuts)")
axes[0].set_xlabel("wmom_T x wmom_T_err")
axes[0].set_yscale("log")

axes[1].hist(shear_table_wl['wmom_T']*shear_table_wl['wmom_T_err'], bins=30)
axes[1].vlines(0.015, 0, 22000, color='red')
axes[1].set_title("T x T_err (after all cuts)")
axes[1].set_xlabel("wmom_T x wmom_T_err")

plt.suptitle("T times T_err", size=16, y=0.98)
plt.savefig('image_outputs_fix/junk2.png', bbox_inches='tight')
plt.show()

### Find Junk Objects

In [ ]:
t_err_line_filt = shear_table_rs_cuts['wmom_T'] > (0.41 - 1.4*shear_table_rs_cuts['wmom_T_err'])
junk_obj_1 = shear_table_rs_cuts[t_err_line_filt]
# t_t_err_filt = (shear_table_rs_cuts['wmom_T'] * shear_table_rs_cuts['wmom_T_err']) > 0.006
# junk_obj_2 = shear_table_rs_cuts[t_t_err_filt]

In [ ]:
len(junk_obj_1)

### Display Junk Object

Code primarily from Robert Lupton's demo notebook: https://github.com/RobertLuptonTheGood/notebooks/blob/2eeee8b9fe35077387485e488c965f1ea3d39418/Demos/Colour%20Images.ipynb

In [ ]:
junk_obj_1[['tract', 'patch_y', 'patch_x', 'ra', 'dec']][100:150]

In [ ]:
tract = 10463
patch = 83

bands = "gri"
images = {}

# get x, y coordinates of object in patch
# filt_loc = junk_obj_1['tract'] == tract
# filt_loc &= junk_obj_1['patch_y'] == 8
# filt_loc &= junk_obj_1['patch_x'] == 3
# filt_loc &= junk_obj_1['shear_type'] == 'ns'
filt_loc = shear_table_wl['tract'] == tract
filt_loc &= shear_table_wl['patch_y'] == 8
filt_loc &= shear_table_wl['patch_x'] == 3
filt_loc &= shear_table_wl['shear_type'] == 'ns'

xys_firefly = []
xys = []

for b in bands:

    coadd = butler.get("deepCoaddCell", 
                    tract=tract, 
                    patch=patch, 
                    band=b, 
                    skymap='lsst_cells_v1', 
                    collections=[cell_collection, cell_collection_g])

    if b=='g':
        bbox = coadd.outer_bbox
        for i, row in shear_table_wl[filt_loc].iterrows():
            radecs = [row['ra'], row['dec']]
            raDec = geom.SpherePoint(row['ra']*geom.degrees, row['dec']*geom.degrees)
            xy_test = geom.PointI(coadd.wcs.skyToPixel(raDec))
            xys_firefly.append(xy_test)
            xy = xy_test - bbox.getBegin()
            xys.append(xy)
            
    images[b] = coadd.stitch().asMaskedImage()
    
    del coadd
    gc.collect()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

for coord in xys:
    circle = plt.Circle((coord.getX(), coord.getY()), 50, color='cyan', fill=False)
    ax.add_patch(circle)

rgb = afwRgb.makeRGB(images['i'], images['r'], images['g'], 9, 20, Q=2)
afwRgb.displayRGB(rgb)
plt.show()

### Look at objects in weak lensing sample

In [ ]:
tract = 10463
patch = 62

bands = "gri"
images = {}

# get x, y coordinates of object in patch
filt_loc = shear_table_wl_ns['tract'] == tract
filt_loc &= shear_table_wl_ns['patch_y'] == 6
filt_loc &= shear_table_wl_ns['patch_x'] == 2

xys_firefly = []
xys = []

for b in bands:

    coadd = butler.get("deepCoaddCell", 
                    tract=tract, 
                    patch=patch, 
                    band=b, 
                    skymap='lsst_cells_v1', 
                    collections=[cell_collection, cell_collection_g])

    if b=='g':
        bbox = coadd.outer_bbox
        for i, row in shear_table_wl_ns[filt_loc].iterrows():
            radecs = [row['ra'], row['dec']]
            raDec = geom.SpherePoint(row['ra']*geom.degrees, row['dec']*geom.degrees)
            xy_test = geom.PointI(coadd.wcs.skyToPixel(raDec))
            xys_firefly.append(xy_test)
            xy = xy_test - bbox.getBegin()
            xys.append(xy)
            
    images[b] = coadd.stitch().asMaskedImage()
    
    del coadd
    gc.collect()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

for coord in xys:
    # circle = plt.Circle((coord.getX()-bbox.getBeginX()+50, coord.getY()-bbox.getBeginY()+50), 50, color='cyan', fill=False)
    circle = plt.Circle((coord.getX(), coord.getY()), 50, color='cyan', fill=False)
    ax.add_patch(circle)

rgb = afwRgb.makeRGB(images['i'], images['r'], images['g'], 4, 15, Q=2)
afwRgb.displayRGB(rgb)
plt.show()

### Try with firefly

In [ ]:
# import lsst.afw.display as afwDisplay
# afwDisplay.setDefaultBackend('firefly')
# display1 = afwDisplay.Display(frame=1)
# display1.getClient().show_lab_tab

# display1.mtv(coadd.stitch().image)
# with display1.Buffering():
#     for coord in xys:
#         display1.dot('o', coord.getX(), coord.getY(), size=50, ctype='orange')

# display1.erase()

## Plot distribution of cell properties in A360 region

Each run of the `get_cell_inputs` function should only take 3-4 minutes. If it's longer, try again later.

In [ ]:
from lsst.sphgeom import Box, HealpixPixelization
import healsparse as hsp
import healpy as hp
from hpgeom import hpgeom
import skyproj

In [ ]:
# define healpix parameters
nside_coverage = 2**8
nside_sparse = 2**14

pixelization = HealpixPixelization(hp.nside2order(nside_sparse))

In [ ]:
# method find the pixel indices that overlap the sky projection of the cell area
def get_cell_pixels(cell, wcs):
    cell_bbox = cell.inner.bbox
    begin_coord = wcs.pixelToSky(cell_bbox.beginX, cell_bbox.beginY)
    end_coord = wcs.pixelToSky(cell_bbox.endX, cell_bbox.endY)
    
    if begin_coord.getRa() < end_coord.getRa():
        ra1 = begin_coord.getRa().asDegrees()
        ra2 = end_coord.getRa().asDegrees()
    else:
        ra1 = end_coord.getRa().asDegrees()
        ra2 = begin_coord.getRa().asDegrees()
    
    if begin_coord.getDec() < end_coord.getDec():
        dec1 = begin_coord.getDec().asDegrees()
        dec2 = end_coord.getDec().asDegrees()
    else:
        dec1 = end_coord.getDec().asDegrees()
        dec2 = begin_coord.getDec().asDegrees()

    indices = hpgeom.query_box(nside=nside_sparse, a0=ra1, a1=ra2, b0=dec1, b1=dec2)
    
    return indices

In [ ]:
# return PSF e magnitude of the cell
def get_psf_e(cell, wcs):
    # get psf image of cell
    psf_im = cell.psf_image
    
    psf_kernel = afwMath.FixedKernel(psf_im)
    psf = meas.KernelPsf(psf_kernel)
    shape = psf.computeShape(psf_im.getBBox().getCenter())
    
    i_xx, i_yy, i_xy = shape.getIxx(), shape.getIyy(), shape.getIxy()
    
    q = Quadrupole(i_xx, i_yy, i_xy)
    s = SeparableDistortionTraceRadius(q)
    
    e1, e2 = s.getE1(), s.getE2()
    e = np.sqrt(e1**2 + e2**2)
    
    return e

In [ ]:
def get_cell_inputs(cell_collection, tract_patch_list, band):

    cell_df = pd.DataFrame()
    cell_ra = []
    cell_dec = []
    pixel_indices = []
    inputs_list = []
    psf_e_list = []

    segs = [] # collection of lines to plot patch outlines

    for tract, patch in tract_patch_list:
    
        coadd = butler.get('deepCoaddCell', 
                         collections = cell_collection, 
                         instrument = 'LSSTComCam', 
                         skymap = 'lsst_cells_v1', 
                         tract = tract, 
                         patch = patch,
                         band = band,)
        # define a wcs from the given coadd
        wcs = coadd.wcs

        # get coadd outline
        coadd_corners = coadd.inner_bbox.getCorners()
    
        for index, corner in enumerate(coadd_corners):
            corner_coord_start = wcs.pixelToSky(corner.getX(), corner.getY())
            if index < 3:
                corner_coord_end = wcs.pixelToSky(coadd_corners[index+1].getX(), coadd_corners[index+1].getY())
            else:
                corner_coord_end = wcs.pixelToSky(coadd_corners[0].getX(), coadd_corners[0].getY())
    
            start_ra = corner_coord_start[0].asDegrees()
            start_dec = corner_coord_start[1].asDegrees()
    
            end_ra = corner_coord_end[0].asDegrees()
            end_dec = corner_coord_end[1].asDegrees()
    
            segs.append(((start_ra, start_dec), (end_ra, end_dec)))
        
        cell_list = list(coadd.cells.keys()) # skips indices that are empty

        # for each cell in cell_list:
        for index, cell_index in enumerate(cell_list):
    
            cell = coadd.cells[cell_index]
    
            # get cell coordinates for removing duplicates
            cell_center = cell.inner.bbox.getCenter()
            cell_center_coord = wcs.pixelToSky(cell_center)
            cell_ra.append(cell_center_coord.getRa().asDegrees())
            cell_dec.append(cell_center_coord.getDec().asDegrees())
        
            pixel_indices.append(get_cell_pixels(cell, wcs))
    
            inputs_list.append(cell.visit_count)
            psf_e_list.append(get_psf_e(cell, wcs))
    
        del coadd
        gc.collect()

    cell_df["ra"] = cell_ra
    cell_df["dec"] = cell_dec
    cell_df["pixels"] = pixel_indices
    cell_df["inputs"] = inputs_list
    cell_df["psf_e"] = psf_e_list

    return cell_df, segs

#### g-band

In [ ]:
cell_df_g, segs_g = get_cell_inputs(cell_collection_g, tract_patch_list, 'g')

In [ ]:
# remove duplicate cells from overlapping patches
cell_df_g = cell_df_g.drop_duplicates(subset=['ra', 'dec'])

In [ ]:
pixel_df_g = cell_df_g.explode('pixels').reset_index(drop=True)
pixel_df_g = pixel_df_g.drop_duplicates(subset=["pixels"]) 
pixel_df_g = pixel_df_g.dropna(subset=['pixels'])

pixels_g = pixel_df_g["pixels"].to_numpy()
pixel_input_g = pixel_df_g["inputs"].to_numpy()
pixel_psf_e_g = pixel_df_g["psf_e"].to_numpy()

In [ ]:
hsp_map_input_g = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.int64)
hsp_map_input_g.update_values_pix(np.array(pixels_g, dtype=np.int64), np.array(pixel_input_g))

hsp_map_psf_e_g = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.float64)
hsp_map_psf_e_g.update_values_pix(np.array(pixels_g, dtype=np.int64), np.array(pixel_psf_e_g))

#### r-band

In [ ]:
cell_df_r, segs_r = get_cell_inputs(cell_collection, tract_patch_list, 'r')

In [ ]:
# remove duplicate cells from overlapping patches
cell_df_r = cell_df_r.drop_duplicates(subset=['ra', 'dec'])

In [ ]:
pixel_df_r = cell_df_r.explode('pixels').reset_index(drop=True)
pixel_df_r = pixel_df_r.drop_duplicates(subset=["pixels"]) 
pixel_df_r = pixel_df_r.dropna(subset=['pixels'])

pixels_r = pixel_df_r["pixels"].to_numpy()
pixel_input_r = pixel_df_r["inputs"].to_numpy()
pixel_psf_e_r = pixel_df_r["psf_e"].to_numpy()

In [ ]:
hsp_map_input_r = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.int64)
hsp_map_input_r.update_values_pix(np.array(pixels_r, dtype=np.int64), np.array(pixel_input_r))

hsp_map_psf_e_r = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.float64)
hsp_map_psf_e_r.update_values_pix(np.array(pixels_r, dtype=np.int64), np.array(pixel_psf_e_r))

#### i-band

In [ ]:
cell_df_i, segs_i = get_cell_inputs(cell_collection, tract_patch_list, 'i')

In [ ]:
# remove duplicate cells from overlapping patches
cell_df_i = cell_df_i.drop_duplicates(subset=['ra', 'dec'])

In [ ]:
pixel_df_i = cell_df_i.explode('pixels').reset_index(drop=True)
pixel_df_i = pixel_df_i.drop_duplicates(subset=["pixels"]) 
pixel_df_i = pixel_df_i.dropna(subset=['pixels'])

pixels_i = pixel_df_i["pixels"].to_numpy()
pixel_input_i = pixel_df_i["inputs"].to_numpy()
pixel_psf_e_i = pixel_df_i["psf_e"].to_numpy()

In [ ]:
hsp_map_input_i = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.int64)
hsp_map_input_i.update_values_pix(np.array(pixels_i, dtype=np.int64), np.array(pixel_input_i))

hsp_map_psf_e_i = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.float64)
hsp_map_psf_e_i.update_values_pix(np.array(pixels_i, dtype=np.int64), np.array(pixel_psf_e_i))

#### Input Distribution Plot

The three missing patches were due to pipeline failures, though are not included within the 0.5 degree radius of the BCG

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 5.3))
sp_g = skyproj.GnomonicSkyproj(ax=ax[0], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_g.draw_hspmap(hsp_map_input_g, vmin=1, vmax=pixel_input_i.max())
sp_g.ax.circle(ra_bcg, dec_bcg, 0.5, color='cyan')
sp_g.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_g.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_g.draw_colorbar(shrink=0.8, label='Number of input warps per cell')
sp_g.ax.set_title("g-band", pad=25)
for seg in segs_g:
    sp_g.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)

sp_r = skyproj.GnomonicSkyproj(ax=ax[1], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_r.draw_hspmap(hsp_map_input_r, vmin=1, vmax=pixel_input_i.max())
sp_r.ax.circle(ra_bcg, dec_bcg, 0.5, color='cyan')
sp_r.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_r.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_r.draw_colorbar(shrink=0.8, label='Number of input warps per cell')
sp_r.ax.set_title("r-band", pad=25)
for seg in segs_r:
    sp_r.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)
    
sp_i = skyproj.GnomonicSkyproj(ax=ax[2], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_i.draw_hspmap(hsp_map_input_i, vmin=1, vmax=pixel_input_i.max())
sp_i.ax.circle(ra_bcg, dec_bcg, 0.5, color='cyan')
sp_i.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_i.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_i.draw_colorbar(shrink=0.8, label='Number of input warps per cell')
sp_i.ax.set_title("i-band", pad=25)
for seg in segs_i:
    sp_i.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)

plt.suptitle("Input Image Distribution of the A360 Region", size=15)

plt.savefig('image_outputs_fix/3_band_image_distribution.png', bbox_inches='tight')
plt.show()

#### PSF Ellipticity Distribution Plot

In [ ]:
# mean PSF ellipticies for each band
print("Mean PSF ellipticity in g-band: ", pixel_psf_e_g.mean())
print("Mean PSF ellipticity in r-band: ", pixel_psf_e_r.mean())
print("Mean PSF ellipticity in i-band: ", pixel_psf_e_i.mean())
print()
# median PSF ellipticies for each band
print("Median PSF ellipticity in g-band: ", np.median(pixel_psf_e_g))
print("Median PSF ellipticity in r-band: ", np.median(pixel_psf_e_r))
print("Median PSF ellipticity in i-band: ", np.median(pixel_psf_e_i))

In [ ]:
# get minimum / maximum PSF ellipticities for a consistent colorbar
print(pixel_psf_e_g.min())
print(pixel_psf_e_r.min())
print(pixel_psf_e_i.min())

print(pixel_psf_e_g.max())
print(pixel_psf_e_r.max())
print(pixel_psf_e_i.max())

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(19.5, 5.5))
sp_g = skyproj.GnomonicSkyproj(ax=ax[0], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_g.draw_hspmap(hsp_map_psf_e_g, vmin=0, vmax=pixel_psf_e_i.max())
sp_g.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_g.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_g.draw_colorbar(shrink=0.9, label='PSF Ellipticity Modulus')
sp_g.ax.set_title("g-band", pad=25)
for seg in segs_g:
    sp_g.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)

sp_r = skyproj.GnomonicSkyproj(ax=ax[1], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_r.draw_hspmap(hsp_map_psf_e_r, vmin=0, vmax=pixel_psf_e_i.max())
sp_r.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_r.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_r.draw_colorbar(shrink=0.9, label='PSF Ellipticity Modulus')
sp_r.ax.set_title("r-band", pad=25)
for seg in segs_r:
    sp_r.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)
    
sp_i = skyproj.GnomonicSkyproj(ax=ax[2], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_i.draw_hspmap(hsp_map_psf_e_i, vmin=0, vmax=pixel_psf_e_i.max())
sp_i.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_i.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_i.draw_colorbar(shrink=0.9, label='PSF Ellipticity Modulus')
sp_i.ax.set_title("i-band", pad=25)
for seg in segs_i:
    sp_i.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)

plt.suptitle("PSF Ellipticity Distribution of the A360 Region", size=15)

plt.savefig('image_outputs_fix/3_band_psf_e_distribution.png', bbox_inches='tight')
plt.show()

## Investigate Patch Failures

In [ ]:
meta_log = 0
meta_log_ids = []
for ref in butler.registry.queryDatasets('metadetectionShear_log', collections=collection):
    meta_log += 1
    meta_log_ids.append(ref.dataId)

    patch_log = butler.get('metadetectionShear_log', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = ref.dataId['tract'], 
                     patch = ref.dataId['patch'],
                     band = 'i')
    
    patch_error = [log['message'] for log in patch_log.model_dump() if log['levelname'] == 'ERROR']
    if len(patch_error) > 0:
        print(patch_error)

## HSM Catalog

HSM catalog after first reading in:  183791 \
HSM catalog after removing RS:  104257 \
HSM source galaxy sample:  24362

In [ ]:
repo_hsm = '/repo/main'
collection_hsm = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'
butler_hsm = Butler(repo_hsm, collections=collection_hsm)

datasetType = 'objectTable'
merged_cat = pd.DataFrame()

skymap = butler_hsm.get('skyMap', skymap='lsst_cells_v1')

# Position of the BCG for A360
ra_bcg = 37.862
dec_bcg = 6.98

# Looking for all patches in delta deg region around it
delta = 0.5
center = geom.SpherePoint(ra_bcg, dec_bcg, geom.degrees)
ra_min, ra_max = ra_bcg - delta, ra_bcg + delta
dec_min, dec_max = dec_bcg - delta, dec_bcg + delta

ra_range = (ra_min, ra_max)
dec_range = (dec_min, dec_max)
radec = [geom.SpherePoint(ra_range[0], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[0], dec_range[1], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[1], geom.degrees)]

tracts_and_patches = skymap.findTractPatchList(radec)

tp_dict = {}
for tract_num in np.arange(len(tracts_and_patches)):
    tract_info = tracts_and_patches[tract_num][0]
    tract_idx = tract_info.getId()
    # All the patches around the cluster
    patches = []
    for i,patch in enumerate(tracts_and_patches[tract_num][1]):
        patch_info = tracts_and_patches[tract_num][1][i]
        patch_idx = patch_info.sequential_index
        patches.append(patch_idx)
    tp_dict.update({tract_idx:patches})

for tract in tp_dict.keys():
    print(f'Loading objects from tract {tract}, patches:{tp_dict[tract]}')
    for patch in tp_dict[tract]:
        dataId = {'tract': tract, 'patch' : patch ,'skymap':'lsst_cells_v1'}
        obj_cat = butler_hsm.get(datasetType, dataId=dataId)
        filt = obj_cat['detect_isPrimary']==True
        filt &= obj_cat['r_cModel_flag']== False
        filt &= obj_cat['i_cModel_flag']== False
        filt &= obj_cat['r_cModelFlux']>0
        filt &= obj_cat['i_cModelFlux']>0
        filt &= obj_cat['refExtendedness'] > 0.5

        merged_cat = pd.concat([merged_cat, obj_cat[filt]], ignore_index=True)

In [ ]:
print("HSM catalog after first reading in: ", len(merged_cat))

HSM catalog after first reading in:  183791

In [ ]:
# RS removal
import astropy.units as u
from astropy.coordinates import SkyCoord

merged_cat = merged_cat.copy()

c1 = SkyCoord(merged_cat['coord_ra'].values*u.deg, merged_cat['coord_dec'].values*u.deg)
c2 = SkyCoord(ra_bcg*u.deg, dec_bcg*u.deg)
sep = c1.separation(c2)

merged_cat['mag_i'] = -2.50 * np.log10(merged_cat['i_cModelFlux']) + 31.4
merged_cat['mag_r'] = -2.50 * np.log10(merged_cat['r_cModelFlux']) + 31.4

color_ri = merged_cat['mag_r'] - merged_cat['mag_i']
rs_hi = 0.6 - (0.1/5.) * (merged_cat['mag_r']-19)
rs_low = 0.4 - (0.1/5.)* (merged_cat['mag_r']-19)

hsm_rs = ((color_ri > rs_low) * (color_ri < rs_hi) * (merged_cat['mag_r'] < 23))

filt = sep.deg < 0.5
filt &= ~hsm_rs

merged_cat_wl = merged_cat[filt]

print("HSM catalog after removing RS: ", len(merged_cat_wl))

HSM catalog after removing RS:  104257

In [ ]:
# Filters to keep sources with good-quality measured shape in i band
source_filt = np.isfinite(merged_cat_wl['i_hsmShapeRegauss_e1'])
source_filt &= np.isfinite(merged_cat_wl['i_hsmShapeRegauss_e2'])
source_filt &= np.sqrt(merged_cat_wl['i_hsmShapeRegauss_e1']**2 + merged_cat_wl['i_hsmShapeRegauss_e2']**2) < 4
source_filt &= merged_cat_wl['i_hsmShapeRegauss_sigma']<= 0.4 
source_filt &= merged_cat_wl['i_hsmShapeRegauss_flag'] == 0
source_filt &= merged_cat_wl['i_blendedness'] < 0.42
source_filt &= merged_cat_wl['i_iPSF_flag']==0

res = 1 - (merged_cat_wl['i_ixxPSF']+ merged_cat_wl['i_iyyPSF']) / (merged_cat_wl['i_ixx']+ merged_cat_wl['i_iyy'])
source_filt &= res >= 0.3

source_filt &= merged_cat['mag_i'] <= 24.5
source_filt &= merged_cat['mag_i'] > 20. # to remove the brightest objects that are likely foreground objects

merged_cat_wl = merged_cat_wl['coord_ra'][source_filt]

print("HSM source galaxy sample: ", len(merged_cat_wl))

## Look at shear profile with no calibration

In [ ]:
# bin tangential and cross shears by radial bins
tan_cross_shears = np.zeros((len(bins_mpc)-1, 2)) # binned tangential and cross shear
mean_mpc = []
tan_errs = []
cross_errs = []

for i in range(0, len(bins_mpc)-1):
    bin_filt_ns = dig_mpc_bins_ns == i+1

    # print number of galaxies in each bin
    print("Rows in bin ", i, " :", len(shear_table_wl_ns['shear_t'][bin_filt_ns]))

    # calulcate mean t and x shears
    mean_g_t = shear_table_wl_ns['shear_t'][bin_filt_ns].mean() # mean g1
    mean_g_x = shear_table_wl_ns['shear_x'][bin_filt_ns].mean() # mean g2

    # calculate errors, 95% confidence interval
    g_t_err = stats.bootstrap([shear_table_wl_ns['shear_t'][bin_filt_ns]], np.mean).confidence_interval
    g_x_err = stats.bootstrap([shear_table_wl_ns['shear_x'][bin_filt_ns]], np.mean).confidence_interval

    tan_err = [g_t_err.low, g_t_err.high]
    cross_err = [g_x_err.low, g_x_err.high]
    
    # apply calibration 
    gs = [mean_g_t, mean_g_x]

    tan_cross_shears[i] = gs
    tan_errs.append(tan_err)
    cross_errs.append(cross_err)

    # get the mean distance from BCG
    mean_deg_sep = shear_table_wl_ns['deg_sep'][bin_filt_ns].mean()
    mean_mpc_sep = cosmo.eval_da(0.22) * mean_deg_sep * np.pi/180
    mean_mpc.append(mean_mpc_sep)

tan_errs = np.array(tan_errs)
cross_errs = np.array(cross_errs)

In [ ]:
galcat = GCData()
galcat['ra'] = shear_table_wl_ns['ra']
galcat['dec'] = shear_table_wl_ns['dec']
galcat['e1'] = shear_table_wl_ns['wmom_g_1'] * 2
galcat['e2'] = shear_table_wl_ns['wmom_g_2'] * 2

galcat['z'] = np.zeros(len(shear_table_wl_ns)) # CLMM needs a redshift column for the source, even if not used

cluster_id = "Abell 360"
gc_object1 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, coordinate_system='euclidean')

gc_object1.compute_galaxy_weights(
        shape_component1="e1",
        shape_component2="e2",
        use_shape_error=False,
        use_shape_noise=True,
        weight_name="w_ls",
        cosmo=cosmo,
        add=True,
    )

moo = clmm.Modeling(massdef="mean", delta_mdef=500, halo_profile_model="nfw")

moo.set_cosmo(cosmo)
moo.set_concentration(4)
moo.set_mass(4.0e14)

z_cl = gc_object1.z

# source properties
# assume sources redshift following a the DESC SRD distribution. This will need updating.

z_distrib_func = utils.redshift_distributions.desc_srd  

# Compute first beta (e.g. eq(6) of WtGIII paper)
beta_kwargs = {
    "z_cl": z_cl,
    "z_inf": 10.0,
    "cosmo": cosmo,
    "z_distrib_func": z_distrib_func,
}
beta_s_mean = utils.compute_beta_s_mean_from_distribution(**beta_kwargs)
beta_s_square_mean = utils.compute_beta_s_square_mean_from_distribution(**beta_kwargs)

rproj = np.logspace(np.log10(0.3),np.log10(7.), 100)

gt_z = moo.eval_reduced_tangential_shear(
    rproj, z_cl, [beta_s_mean, beta_s_square_mean], z_src_info="beta", approx="order2"
)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(8,6))

point_size = 40
point_alpha = 1

# g1 calibrated
axes.scatter(mean_mpc, tan_cross_shears[:,0],
             marker='.', s=point_size, label='tangential shear')
axes.plot(mean_mpc, tan_cross_shears[:,0], '-o')
axes.vlines(mean_mpc, tan_errs[:,0], tan_errs[:,1], colors='blue')

cross_axis = np.add(mean_mpc, 0.05) # add offset to visually differentiate

axes.scatter(cross_axis, tan_cross_shears[:,1],
             marker='.', s=point_size, label='cross shear')
axes.plot(cross_axis, tan_cross_shears[:,1], '-o')
axes.vlines(cross_axis, cross_errs[:,0], cross_errs[:,1], colors='orange')

plt.axhline(0.0, color='k', ls=':')

plt.plot(rproj, gt_z, label='NFW (model, not fit), M500c=4e14 Msun, c=4, n(z)=SRD', ls=':')

plt.xscale('log')
plt.axhline(0.0, color='k', ls=':')
# plt.ylim([-0.03,0.08])
plt.ylim([-0.07,0.09])
plt.xlim([0.3,7])
#plt.yscale('log')
axes.set_xlabel('R [Mpc]')
axes.set_ylabel("reduced shear")
axes.legend(loc=1)

plt.savefig('image_outputs_fix/shear-no-cal.png', bbox_inches='tight')
plt.show()